In [17]:
import pandas as pd
import networkx as nx
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# 读取节点文件和边文件
nodes_df = pd.read_csv('data/team1/Node.csv')
edges_df = pd.read_csv('data/team1/Link.csv')


In [18]:
# 创建空的无向图
G = nx.Graph()

# 添加节点到图中
for i, row in nodes_df.iterrows():
    G.add_node(row['id'], label=row['type'])

# 添加边到图中
for i, row in edges_df.iterrows():
    G.add_edge(row['source'], row['target'], label=row['relation'])

# 计算力导向布局
pos = nx.spring_layout(G)

In [19]:
# 提取位置信息
node_positions = {node: pos[node] for node in G.nodes()}

# 创建 scatter plot
fig = px.scatter(x=[pos[x][0] for x in G.nodes()],
                 y=[pos[x][1] for x in G.nodes()],
                 # text=[nodes_df.loc[nodes_df['id'] == x, 'type'].iloc[0] for x in G.nodes()],
                 color=[nodes_df.loc[nodes_df['id'] == x, 'type'].iloc[0] for x in G.nodes()],
                 labels={'text': 'Node'},
                 title='Force-Directed Network Graph')

# 隐藏文字
# fig.update_traces(textfont=dict(color='rgba(0,0,0,0)'))

fig.show()

In [21]:
# 添加边到 scatter plot
for edge in G.edges():
    fig.add_scatter(x=[pos[edge[0]][0], pos[edge[1]][0], None],
                    y=[pos[edge[0]][1], pos[edge[1]][1], None],
                    mode='lines',
                    line=dict(color='blue', width=0.1))

fig.update_traces(marker=dict(size=8))

fig.show()

In [23]:
import dash_cytoscape as cyto
from dash import Dash, html

In [24]:
app = Dash(__name__)

# 创建空的无向图
G = nx.Graph()

# 添加节点到图中
for i, row in nodes_df.iterrows():
    G.add_node(str(row['id']), label=str(row['type']))

# 添加边到图中
for i, row in edges_df.iterrows():
    G.add_edge(str(row['source']), str(row['target']))

# 使用 NetworkX 的布局算法计算节点位置
pos = nx.spring_layout(G)

# 创建 cytoscape 元素列表
elements = []

# 添加节点到元素列表，并使用计算的位置信息
for node_id, node_attrs in G.nodes(data=True):
    elements.append({
        'data': {'id': node_id, 'label': node_attrs['label']},
        'position': {'x': pos[node_id][0] * 500, 'y': pos[node_id][1] * 500}  # 乘以 500 是为了调整布局范围
    })

# 添加边到元素列表
for u, v in G.edges():
    elements.append({
        'data': {'source': u, 'target': v}
    })


# 创建 cytoscape 组件
cyto_graph = cyto.Cytoscape(
    id='cytoscape-two-nodes',
    layout={'name': 'preset'},
    style={'width': '100%', 'height': '400px'},
    elements=elements
)

# 在布局中添加 cytoscape 组件
app.layout = html.Div([
    cyto_graph
])